# Streaming
(Adapting notebook 16 from the https://be189.github.io/ course.)

- Uses the `gui_data` websocket  the GUI Graph example needs to be running in Bela and the GUI window needs to be open (?)
- The asleep async value needs to be adjusted with the plot refresh rate so that python has bandwidth to receive data + update plot stream
- The dynamic streaming plot works fine, but when trying to add widgets and their callbacks it becomes very convoluted since the receiving data task runs in async and sending events to start and stop receiving is complex. It would be easier to send messages back to Bela to start and stop streaming. 


In [ ]:
import re
import asyncio
import time
import array
import copy

import numpy as np
import pandas as pd

import serial
import serial.tools.list_ports
from  websockets.sync.client import connect

import bokeh.plotting
import bokeh.io
import bokeh.driving

from bokeh.resources import INLINE
bokeh.io.output_notebook(INLINE)

notebook_url = "localhost:8888"

import os
# os.environ["BOKEH_ALLOW_WS_ORIGIN"] = notebook_url
os.environ["BOKEH_ALLOW_WS_ORIGIN"] ="1t4j54lsdj67h02ol8hionopt4k7b7ngd9483l5q5pagr3j2droq"

In [ ]:
# use with Gui Graph example

ip = "192.168.7.2"
port=5555
address = "gui_data"
notebook_url = "localhost:8888"
websocket =  connect(f'ws://{ip}:{port}/{address}')

## Dynamic plots (no widgets)

Bela sends a continuous stream of messages. `daq_stream_async` is an async task that receives and parses the messages into the `data` dictionary. `potentiometer_app` accesses this dictionary and feeds it to a bokeh plot that gets dynamically updated. 

The streaming can last until the task is cancelled with `daq_task.cancel()` or run until it collects `n_data` points. 




In [ ]:
async def daq_stream_async(data,
                           websocket,
                           n_data = 0, # if n_data is 0, stream until task is cancelled
                           buffer_size = 100000): # if n_data is 0, keep onl last buffer_size points so that size of tmp_data does not grow with time
    
    tmp_data = copy.deepcopy(data)
    channel = None
    
    if n_data != 0:
        buffer_size = n_data

        
    while True if n_data == 0 else len(data["time_ms"]) < n_data or len(data["voltage"]) < n_data : 

        msg =  websocket.recv()

        if len(msg) == 3:
            channel = int(str(msg)[2])
        else:
            # first channel to come in has to be 0, otherwise taking data from different render iterations
            arr = array.array('f', msg).tolist()        
            if channel == 0:
                tmp_data['time_ms'] += arr
            elif channel == 1:
                tmp_data['voltage'] += arr    
                
            # update time_ms and voltage in one go to avoid feeding data of different lengths to streaming plot
            if len(tmp_data['time_ms']) == len(tmp_data['voltage']):
                
                # if n_data is not 0, drop extra points; if n_data is 0, keep last buffer_size points
                
                if n_data != 0: # drop extra points 
                    # data = copy.deepcopy(tmp_data) # this is not working -- bokeh plot doesn't show anything
                    tmp_data["time_ms"] = tmp_data["time_ms"][:n_data]
                    tmp_data["voltage"] = tmp_data["voltage"][:n_data]

                elif n_data == 0 and len(tmp_data["time_ms"]) > buffer_size: # max length for data 
                    tmp_data["time_ms"] = tmp_data["time_ms"][-buffer_size:]
                    tmp_data["voltage"] = tmp_data["voltage"][-buffer_size:]
                
                data["time_ms"] = copy.deepcopy(tmp_data["time_ms"])
                data["voltage"] = copy.deepcopy(tmp_data["voltage"])
                
            
            await asyncio.sleep(0.01) # this value? -- adjust to refresh streaming?

    
    return dict({
        "time_ms": data["time_ms"],
        "voltage": data["voltage"]
    })    

In [ ]:
def potentiometer_app(data, 
                      n_data=0,       # if n_data is 0, stream until task is cancelled, otherwise stream until n_data points are stored in data
                      rollover=None,  # amount of data displayed on plot
                      plot_update_delay=90):
    """Return a function defining a Bokeh app for streaming
    data up to `n_data` data points. A maximum of `rollover`
    data points are shown at a time.
    """
    def _app(doc):
        # Instatiate figures
        p = bokeh.plotting.figure(
            frame_width=500,
            frame_height=175,
            x_axis_label="time (s)",
            y_axis_label="voltage (V)",
            y_range=[-0.2, 5.2],
        )

        # No padding on x_range makes data flush with end of plot
        p.x_range.range_padding = 0

        # Start with an empty column data source with time and voltage
        source = bokeh.models.ColumnDataSource({"t": [], "V": []})

        # Put a line glyph
        r = p.line(source=source, x="t", y="V")

        @bokeh.driving.linear()
        def update(step):
            # Shut off periodic callback if we have plotted all of the data
            if n_data > 0 and step > n_data:
                doc.remove_periodic_callback(pc)
            else:
                # Update plot by streaming in data
                source.stream(
                    {
                        "t": np.array(data['time_ms']) / 1000,
                        "V": data['voltage'],
                    },
                    rollover,
                )

        doc.add_root(p)
        pc = doc.add_periodic_callback(update, plot_update_delay)
        
        
    return _app

In [ ]:
data = dict(
        time_ms = [],
        voltage = []
    )

n_data = 5000
n_data = 0 # stream until task is cancelled

rollover = 1000 # number of points shown in plot

bokeh.io.show(potentiometer_app(data,n_data=n_data, plot_update_delay=50, rollover=rollover), notebook_url=notebook_url)
daq_task = asyncio.create_task(daq_stream_async(data,websocket, n_data = n_data))

In [ ]:
daq_task.cancel()

In [ ]:
daq_task.done()


## Dynamic plots with widgets (needs Bela side)
We can add widgets to the previous plot to allow to start/stopping the streaming, freeze the plot, save the data in a csv file, etc. Building this without communicating back with Bela is convoluted because we need to constantly interact with the async task – this will be easier to do once we can send messages back to Bela through websockets.

In [ ]:
trigger_stream = asyncio.Event()
trigger_stream.set()
trigger_stream.is_set()

In [ ]:
trigger_stream.clear()
trigger_stream.is_set()

In [ ]:
async def daq_stream_async(data,
                           websocket,
                           n_data = 0, # if n_data is 0, stream until task is cancelled
                           buffer_size = 100000): # if n_data is 0, keep onl last buffer_size points so that size of tmp_data does not grow with time
    
    tmp_data = copy.deepcopy(data)
    channel = None
    
    if n_data != 0:
        buffer_size = n_data

        
    while (True if n_data == 0 else len(data["time_ms"]) < n_data or len(data["voltage"]) < n_data) :     

        msg =  websocket.recv()

        if len(msg) == 3:
            channel = int(str(msg)[2])
        else:
            # first channel to come in has to be 0, otherwise taking data from different render iterations
            arr = array.array('f', msg).tolist()        
            if channel == 0:
                tmp_data['time_ms'] += arr
            elif channel == 1:
                tmp_data['voltage'] += arr    
                
            # update time_ms and voltage in one go to avoid feeding data of different lengths to streaming plot
            if len(tmp_data['time_ms']) == len(tmp_data['voltage']):
                
                # if n_data is not 0, drop extra points; if n_data is 0, keep last buffer_size points
                
                if n_data != 0: # drop extra points 
                    # data = copy.deepcopy(tmp_data) # this is not working -- bokeh plot doesn't show anything
                    tmp_data["time_ms"] = tmp_data["time_ms"][:n_data]
                    tmp_data["voltage"] = tmp_data["voltage"][:n_data]

                elif n_data == 0 and len(tmp_data["time_ms"]) > buffer_size: # max length for data 
                    tmp_data["time_ms"] = tmp_data["time_ms"][-buffer_size:]
                    tmp_data["voltage"] = tmp_data["voltage"][-buffer_size:]
                
                data["time_ms"] = copy.deepcopy(tmp_data["time_ms"])
                data["voltage"] = copy.deepcopy(tmp_data["voltage"])
                
            
            await asyncio.sleep(0.01) # this value? -- adjust to refresh streaming?

    
    return dict({
        "time_ms": data["time_ms"],
        "voltage": data["voltage"]
    })    

### layout

In [ ]:
def plot():
    """Build a plot of voltage vs time data"""
    # Set up plot area
    p = bokeh.plotting.figure(
        frame_width=500,
        frame_height=175,
        x_axis_label="time (s)",
        y_axis_label="voltage (V)",
        title="streaming data",
        y_range=[-0.2, 5.2],
        toolbar_location="above",
    )

    # No range padding on x: signal spans whole plot
    p.x_range.range_padding = 0

    # We'll sue whitesmoke backgrounds
    p.border_fill_color = "whitesmoke"

    # Defined the data source
    source = bokeh.models.ColumnDataSource(data=dict(t=[], V=[]))

    # If we are in streaming mode, use a line, dots for on-demand
    p.line(source=source, x="t", y="V")

    return p, source

def controls():
    stream = bokeh.models.Toggle(label="Stream", button_type="success", width=100)
    save_notice = bokeh.models.Div(text="<p>No streaming data saved</p>", width=165)
    save = bokeh.models.Button(label="save", button_type="primary", width=100)
    reset = bokeh.models.Button(label="reset", button_type="warning", width=100)
    file_input = bokeh.models.TextInput(title="file name", value="data.csv", width=165)
    
    return dict (
        stream = stream,
        reset = reset,
        save = save,
        file_input = file_input,
        save_notice = save_notice
    )

def layout(p, ctrls):
    buttons = bokeh.layouts.row(
        bokeh.models.Spacer(width=30),
        ctrls["stream"],
        bokeh.models.Spacer(width=295),
        ctrls["reset"],
    )
    left = bokeh.layouts.column(p, buttons, spacing=15)
    right = bokeh.layouts.column(
        bokeh.models.Spacer(height=50),
        ctrls["file_input"],
        ctrls["save"],
        ctrls["save_notice"],
    )
    return bokeh.layouts.row(
        left, right, spacing=30, margin=(30, 30, 30, 30), background="whitesmoke",
    )

In [ ]:
p, source = plot()
ctrls = controls()
bokeh.io.show(layout(p, ctrls))

### callbacks

In [ ]:
def stream_callback(): ## ??
    # trigger_stream.set()
    # if new:
    #     stream_data["mode"] = "stream"
    # else:
    #     stream_data["mode"] = "on-demand"
    #     arduino.write(bytes([ON_REQUEST]))

    # arduino.reset_input_buffer()
    pass

def reset_callback(data, source):

    # Black out the data dictionaries
    data["time_ms"] = []
    data["voltage"] = []

    # Reset the sources
    source.data = dict(t=[], V=[])
    
    
def save_callback(data, controls):
    # Convert data to data frame and save
    df = pd.DataFrame(data={"time (ms)": data["time_ms"], "voltage (V)": data["voltage"]})
    df.to_csv(controls["file_input"].value, index=False)

    # Update notice text
    notice_text = "<p>" + "Streaming"
    notice_text += f" data was last saved to {controls['file_input'].value}.</p>"
    controls["save_notice"].text = notice_text
    
def disable_controls(controls):
    """Disable all controls."""
    for key in controls:
        controls[key].disabled = True


def shutdown_callback(daq_task, controls):
    # Disable controls
    disable_controls(controls)

    # Stop DAQ async task
    daq_task.cancel()



def stream_update(data, source, rollover):
    # Update plot by streaming in data
    new_data = {
        "t": np.array(data["time_ms"]) / 1000,
        "V": data["voltage"],
    }
    source.stream(new_data, rollover)

In [ ]:
def potentiometer_app(
    data, daq_task, rollover=400, stream_plot_delay=90,
):
    def _app(doc):
        # Plots
        p_stream, stream_source = plot()

        # Controls
        stream_controls = controls()

        # Shut down
        shutdown_button = bokeh.models.Button(
            label="shut down", button_type="danger", width=100
        )

        # Layouts
        stream_layout = layout(p_stream, stream_controls)

        # Shut down layout
        shutdown_layout = bokeh.layouts.row(
            bokeh.models.Spacer(width=675), shutdown_button
        )

        app_layout = bokeh.layouts.column(
            stream_layout, shutdown_layout
        )

        def _stream_callback(attr, old, new):
            # streaming starts automatically -- is there a way to trigger this??
            # stream_callback(arduino, data, new)
            return
        
        def _stream_reset_callback(event=None):
            reset_callback(
                data,
                stream_source,
            )


        def _stream_save_callback(event=None):
            save_callback( data, stream_controls)

        def _shutdown_callback(event=None):
            shutdown_callback(
                daq_task, stream_controls
            )

        @bokeh.driving.linear()
        def _stream_update(step):
            stream_update(data, stream_source, rollover)

            # # Shut down server if Arduino disconnects (commented out in Jupyter notebook)
            # if not arduino.is_open:
            #     sys.exit()

        # Link callbacks
        stream_controls["stream"].on_change("active", _stream_callback)
        stream_controls["reset"].on_click(_stream_reset_callback)
        stream_controls["save"].on_click(_stream_save_callback)
        shutdown_button.on_click(_shutdown_callback)

        # Add the layout to the app
        doc.add_root(app_layout)

        # Add a periodic callback, monitor changes in stream data
        pc = doc.add_periodic_callback(_stream_update, stream_plot_delay)

    return _app

In [ ]:
data = dict(
        time_ms = [],
        voltage = []
    )
n_data = 1000
daq_task = asyncio.create_task(daq_stream_async(data,websocket, n_data = n_data))
bokeh.io.show(
    potentiometer_app(data, daq_task),
    notebook_url=notebook_url,
)

In [ ]:
daq_task.cancel()